In [7]:
#import matplotlib.pyplot as plt
#import seaborn as sns


import pandas as pd
import numpy as np
import random as rnd

from sklearn.cross_validation import KFold, cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train_=pd.read_csv('../train.csv')
validate_=pd.read_csv('../validate.csv')
#test=pd.read_csv('../testwDSM.csv')

train_.shape, validate_.shape, #test.shape

((1478761, 33), (728346, 33))

In [8]:
train = train_.query('DSMCRIT < 14')
validate = validate_.query('DSMCRIT < 14')
print train['DSMCRIT'].value_counts()
print train.shape

4     324606
5     270924
7     144364
8     143138
3     135811
9     108672
10    103346
6      82922
2      45191
13     20397
11     17281
1      14222
12     12852
0      11595
Name: DSMCRIT, dtype: int64
(1435321, 33)


In [9]:
#alcohol
print train['DSMCRIT'].value_counts() / train['DSMCRIT'].count()
print train['SUB1'].value_counts() / train['SUB1'].count()

4     0.226156
5     0.188755
7     0.100580
8     0.099725
3     0.094621
9     0.075713
10    0.072002
6     0.057772
2     0.031485
13    0.014211
11    0.012040
1     0.009909
12    0.008954
0     0.008078
Name: DSMCRIT, dtype: float64
2     0.425528
4     0.193937
7     0.113757
5     0.108025
3     0.073880
10    0.051074
13    0.009122
20    0.005884
11    0.005407
6     0.004178
16    0.001860
8     0.001654
9     0.001152
18    0.001124
1     0.001055
17    0.000904
12    0.000612
15    0.000607
14    0.000240
Name: SUB1, dtype: float64


In [10]:
#
train.query('SUB1 == 4')['DSMCRIT'].value_counts() / train.query('SUB1 == 4')['DSMCRIT'].count()

7     0.470344
10    0.343247
8     0.050190
6     0.031621
13    0.024170
2     0.020416
4     0.018422
9     0.015480
5     0.011564
0     0.005989
11    0.005762
12    0.001739
1     0.000762
3     0.000295
Name: DSMCRIT, dtype: float64

In [11]:
#train.describe()
train = train.sample(20000)
validate = validate.sample(6000)
train.shape, validate.shape, validate.head(2)

((20000, 33),
 (6000, 33),
         AGE  GENDER  RACE  ETHNIC  MARSTAT  EDUC  EMPLOY  DETNLF  PREG  VET  \
 280713    6       1     5       5        1     3       1       0     2    2   
 294923   12       2     2       5        4     4       3       0     2    2   
 
          ...     FREQ2  FRSTUSE2  SUB3  ROUTE3  FREQ3  FRSTUSE3  NUMSUBS  IDU  \
 280713   ...         0         0     1       0      0         0        1    1   
 294923   ...         0         0     1       0      0         0        1    0   
 
         DSMCRIT  PSYPROB  
 280713        8        2  
 294923        4        1  
 
 [2 rows x 33 columns])

In [28]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['DSMCRIT',  #'NUMSUBS'
             ]
drop_list_select = ['RACE', 'PREG', 'ARRESTS', 'PSYPROB', 'DETNLF', 'ETHNIC', 'MARSTAT', 'GENDER', 'EDUC'
                   ,'LIVARAG', 'EMPLOY', 'SUB3']
X_train = train.drop(drop_list + drop_list_select, axis=1)
Y_train = train["DSMCRIT"]
#X_validate = validate.drop(drop_list + drop_list_select, axis=1)
#Y_validate = validate["DSMCRIT"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, #X_validate.shape, #X_test.shape



((20000, 20), (6000, 20))

In [29]:
print X_train.columns.tolist()

['AGE', 'VET', 'SERVSETA', 'METHUSE', 'PSOURCE', 'DETCRIM', 'NOPRIOR', 'SUB1', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'SUB2', 'ROUTE2', 'FREQ2', 'FRSTUSE2', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'NUMSUBS', 'IDU']


In [30]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif,chi2
#Selector_f = SelectPercentile(f_classif, percentile=25)
Selector_f = SelectKBest(f_classif, k=10)
Selector_f.fit(X_train,Y_train)

zipped = zip(X_train.columns.tolist(),Selector_f.scores_)
ans = sorted(zipped, key=lambda x: x[1])
for n,s in ans:
     print 'F-score: %3.2ft for feature %s' % (s,n)
        
        
X_train= SelectKBest(f_classif, k=10).fit_transform(X_train, Y_train)


F-score: 63.27t for feature ROUTE3
F-score: 64.95t for feature ROUTE2
F-score: 65.26t for feature VET
F-score: 83.68t for feature SUB2
F-score: 98.72t for feature FRSTUSE2
F-score: 99.60t for feature FRSTUSE3
F-score: 101.65t for feature FREQ2
F-score: 102.74t for feature FREQ3
F-score: 110.86t for feature DETCRIM
F-score: 188.05t for feature FREQ1
F-score: 190.67t for feature NUMSUBS
F-score: 208.75t for feature NOPRIOR
F-score: 220.19t for feature PSOURCE
F-score: 285.62t for feature METHUSE
F-score: 350.07t for feature ROUTE1
F-score: 372.02t for feature AGE
F-score: 411.05t for feature FRSTUSE1
F-score: 501.99t for feature IDU
F-score: 869.04t for feature SERVSETA
F-score: 1520.28t for feature SUB1


In [33]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
X_train = onehotlabels

#onehotlabels = enc.transform(X_validate).toarray()
#X_validate = onehotlabels

X_train.shape, #X_validate.shape

((20000, 304),)

In [34]:
# Logistic Regression
logreg = LogisticRegression(n_jobs=-1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)

l_acc_log = cross_val_score(logreg, X_train, Y_train, cv=5)
acc_log = round(np.mean(l_acc_log), 3)

l_acc_log = ['%.3f' % elem for elem in l_acc_log]
print l_acc_log
print acc_log

['0.680', '0.687', '0.669', '0.682', '0.686']
0.681


In [35]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=200, max_depth=20, n_jobs=-1)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)

l_acc_random_forest = cross_val_score(random_forest, X_train, Y_train, cv=5)
acc_random_forest = round(np.mean(l_acc_random_forest), 3)

l_acc_random_forest = ['%.3f' % elem for elem in l_acc_random_forest]
print l_acc_random_forest
print acc_random_forest

['0.665', '0.674', '0.651', '0.663', '0.665']
0.664


In [36]:
# Linear SVC
linear_svc = LinearSVC(C=1.0)
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)

l_acc_linear_svc = cross_val_score(linear_svc, X_train, Y_train, cv=5)
acc_linear_svc = round(np.mean(l_acc_linear_svc), 3)
l_acc_linear_svc = ['%.3f' % elem for elem in l_acc_linear_svc]
print l_acc_linear_svc
print acc_linear_svc

['0.675', '0.684', '0.666', '0.679', '0.680']
0.677


In [37]:
print 'predict-sub2-woflags-newsplit-sample20000'
models = pd.DataFrame({
    'Model': ['Logistic Regression', 
              'Random Forest','Linear SVC'],
    'Cross Validation': [l_acc_log, 
              l_acc_random_forest, l_acc_linear_svc],
    'Cross Validation Mean': [acc_log, 
              acc_random_forest, acc_linear_svc]
    })
models.sort_values(by='Cross Validation Mean', ascending=False)


predict-sub2-woflags-newsplit-sample20000


,Cross Validation,Cross Validation Mean,Model
0,"[0.680, 0.687, 0.669, 0.682, 0.686]",0.681,Logistic Regression
2,"[0.675, 0.684, 0.666, 0.679, 0.680]",0.677,Linear SVC
1,"[0.665, 0.674, 0.651, 0.663, 0.665]",0.664,Random Forest
